In [2]:
from rlcms.composites import Composite
import ee
import geemap

ee.Initialize(project='sig-ee-cloud')

In [3]:
# some args we'll use in both composites
start_date = '2018-01-01'
end_date = '2018-12-31'
reducer = 'mean' # min, max, ee.Reducer objects
composite_mode = 'annual' # 'annual'

sesheke = ee.FeatureCollection("projects/sig-ee-cloud/assets/rlcmsTests/sesheke")

s1composite = Composite(dataset='Sentinel1',
                                  region=sesheke,
                                  start_date=start_date,
                                  end_date=end_date,
                                 composite_mode=composite_mode,
                                 reducer=reducer)
s2composite = Composite(dataset='Sentinel2',
                                 region=sesheke,
                                 start_date=start_date,
                                 end_date=end_date,
                                 indices=['IBI','SAVI'],
                                 composite_mode=composite_mode,
                                 reducer=reducer)
# look at the Composite object
print(s1composite.__dict__)

# retrieve band names
print(f"S1 Composite bands:{s1composite.bands}")
print(f"S2 Composite bands: {s2composite.bands}")

# retrieve ee.Image from Composite object 
s1composite_image = s1composite.image
s2composite_image = s2composite.image



{'dataset': 'Sentinel1', 'region': [[[24.280821946182392, -17.49195652431585], [24.31515256390456, -17.49195647837019], [24.31515256390456, -17.467721268955287], [24.280821946182392, -17.467721310351774], [24.280821946182392, -17.49195652431585]]], 'start_date': '2018-01-01', 'end_date': '2018-12-31', 'composite_mode': 'annual', 'reducer': 'mean', 'bands': ['VV_mean', 'VH_mean', 'angle_mean'], 'image': <ee.image.Image object at 0x00000240BF98E5E0>}
S1 Composite bands:['VV_mean', 'VH_mean', 'angle_mean']
S2 Composite bands: ['blue_mean', 'green_mean', 'red_mean', 'nir_mean', 'swir1_mean', 'swir2_mean', 'IBI_mean', 'SAVI_mean']


In [4]:
Map = geemap.Map()

Map.addLayer(s1composite_image.select(0,1),{'min':-20,'max':20},'S1 Composite VV,VH')
Map.addLayer(s2composite_image.select(0,1,2),{'min':0,'max':2500},'S2 Composite first 3')
Map.centerObject(sesheke,12)

Map

Map(center=[-17.479839363154916, 24.297987254521022], controls=(WidgetControl(options=['position', 'transparen…

In [5]:
# stack two Composites together
all_composite = s1composite.stack(s2composite)
print(all_composite.bandNames().getInfo())

['Sentinel1_VV_mean', 'Sentinel1_VH_mean', 'Sentinel1_angle_mean', 'Sentinel2_blue_mean', 'Sentinel2_green_mean', 'Sentinel2_red_mean', 'Sentinel2_nir_mean', 'Sentinel2_swir1_mean', 'Sentinel2_swir2_mean', 'Sentinel2_IBI_mean', 'Sentinel2_SAVI_mean']


In [6]:
Map = geemap.Map()
viz = {'bands':['Sentinel2_nir_mean','Sentinel2_green_mean','Sentinel2_blue_mean'],'min':0,'max':2500}
Map.addLayer(all_composite,viz,'all_composite S2 false composite')
Map.centerObject(all_composite,12)
Map

Map(center=[-17.479839363154916, 24.297987254521022], controls=(WidgetControl(options=['position', 'transparen…

In [8]:
# Export composite to GEE Asset
from rlcms.utils import export_img_to_asset
asset_id = '<your/assetid/here>'
export_img_to_asset(image=all_composite,
                 description='exportAllCompositeSesheke',
                 assetId=asset_id,
                 region=all_composite.geometry(),
                 scale=10)

Export Started (Asset): projects/sig-ee-cloud/assets/rlcmsTests/seshekeS1S2Composite
